<a href="https://colab.research.google.com/github/HarshithaMulemane/acm-github-temp/blob/main/SummaryOfResearchPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pymupdf openai pandas pypdf2

In [17]:
pip install google-generativeai pandas PyMuPDF

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import google.generativeai as genai
import fitz  # PyMuPDF for PDF processing
import os
import pandas as pd
import re

# ✅ Configure Google Gemini API
genai.configure(api_key="")

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

def summarize_text_with_gemini(text):
    """Summarizes text and extracts specific details using the Gemini 1.5 Pro model."""
    model = genai.GenerativeModel("gemini-1.5-pro-latest")  # ✅ Best available model

    prompt = f"""
    Extract and summarize the following details from the research paper:
    - Title
    - Objective
    - Algorithm
    - Mobility
    - Network Environment
    - Results
    - Future Works
    - Signal Propagation Characteristics
    - Author's Affiliation

    Text: {text}
    """

    try:
        response = model.generate_content(prompt)
        return response.text if response else "No response received."
    except Exception as e:
        return f"Error: {str(e)}"

def clean_text(text):
    """Removes asterisks and unnecessary formatting from extracted text."""
    return re.sub(r'\*+\s*', '', text).strip()

# 🔹 Set folder path for PDFs
pdf_folder = "/content/drive/MyDrive/Test"

data_list = []

for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"Processing: {filename}")

        text = extract_text_from_pdf(pdf_path)
        summary = summarize_text_with_gemini(text)

        # Split and clean the summary into the required fields
        try:
            title = clean_text(summary.split("Objective:")[0])
            objective = clean_text(summary.split("Objective:")[1].split("Algorithm:")[0])
            algorithm = clean_text(summary.split("Algorithm:")[1].split("Mobility:")[0])
            mobility = clean_text(summary.split("Mobility:")[1].split("Network Environment:")[0])
            network_env = clean_text(summary.split("Network Environment:")[1].split("Results:")[0])
            results = clean_text(summary.split("Results:")[1].split("Future Works:")[0])
            future_works = clean_text(summary.split("Future Works:")[1].split("Signal Propagation Characteristics:")[0])
            signal_prop = clean_text(summary.split("Signal Propagation Characteristics:")[1].split("Author's Affiliation:")[0])
            author_aff = clean_text(summary.split("Author's Affiliation:")[1])

            # Add data to the list
            data_list.append([title, objective, algorithm, mobility, network_env, results, future_works, signal_prop, author_aff])
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

# Create a DataFrame with the specified column names
columns = ["Title", "Objective", "Algorithm", "Mobility", "Network Environment", "Results", "Future Works", "Signal Propagation Characteristics", "Author's Affiliation"]
df = pd.DataFrame(data_list, columns=columns)

# Save as Excel file
output_path = "/content/drive/MyDrive/ResearchPaper_Summary7.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ Summary saved to {output_path}")


Processing: Copy of Blockchain Enabled SDN Framework for Security Management in 5G Applications.pdf
